In [90]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [91]:
def match_features_ratio(img0,img1):
    detector = cv2.ORB_create(100) # at most 100 descriptors
    kps0, desc0 = detector.detectAndCompute(img0, None)
    kps1, desc1 = detector.detectAndCompute(img1, None)
    matcher = cv2.BFMatcher_create(cv2.NORM_HAMMING, False)
    
    try:
        matches01 = matcher.knnMatch(desc0, desc1, k=2)
        matches10 = matcher.knnMatch(desc1, desc0, k=2)
    except:
        return []
    
    def ratio_test(matches, thr):
    # Input: List of matches and threshold
    # Output: List of "good" matches
        good_matches = []
        for m in matches:
            ratio = m[0].distance/m[1].distance # Best match distance / Second best match distance
            if ratio < thr:
                good_matches.append(m[0]) # Keep the best match
        return good_matches

    thr = 0.825
    good_matches01 = ratio_test(matches01,thr)
    good_matches10 = ratio_test(matches10, thr)
    
    good_matches10_ = [(m.trainIdx, m.queryIdx) for m in good_matches10]
    final_matches = [m for m in good_matches01 if (m.queryIdx, m.trainIdx) in good_matches10_]

    points = np.array([kps0[m.queryIdx].pt for m in final_matches], np.uint8)
    return points


In [92]:
def match_features_homography(img0, img1):
    detector = cv2.ORB_create(50)
    kps0, fea0 = detector.detectAndCompute(img0, None)
    kps1, fea1 = detector.detectAndCompute(img1, None)
    matcher = cv2.BFMatcher_create(cv2.NORM_HAMMING, False)
    matches = matcher.match(fea0, fea1)
    pts0 = np.float32([kps0[m.queryIdx].pt for m in matches]).reshape(-1,2)
    pts1 = np.float32([kps1[m.trainIdx].pt for m in matches]).reshape(-1,2)
    H, mask = cv2.findHomography(pts0, pts1, cv2.RANSAC, 3.0) #last argument = internal RANSAC param
    
    final_matches = [m for i,m in enumerate(matches) if mask[i]]
    worst_match = sorted([m.distance for m in final_matches])[-1]
    
    if worst_match < 100:
        points = np.array([kps0[m.queryIdx].pt for m in final_matches], np.uint8)
    else:
        points = []
    return points

In [93]:
img0 = cv2.imread("../opencv-np-workshop/data/img/nemo.jpg")
img1 = cv2.imread("../opencv-np-workshop/data/img/nemo-sea.jpg")
match_features_homography(img0,img1)

[]

In [94]:
def calc_rect(points):
    xx, yy = zip(*points)
    min_x = min(xx); min_y = min(yy); max_x = max(xx); max_y = max(yy)
    return (min_x, min_y), (max_x, max_y)

In [95]:
def video_detector(template,video,finder):
    img0 = cv2.imread(template, 0)
    cap = cv2.VideoCapture(video)
    i = 0
    ret = True
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            frame2gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            try:
                points = finder(img0, frame2gray)
                if points.shape[0]>4:
                    i+=1 
                    top_left, bottom_right = calc_rect(points)                
                    cv2.rectangle(frame, top_left,bottom_right,(0,255,0),5)
                    cv2.imwrite("../DEMO/nemo/abc_"+str(i+1)+'.png',frame)
            except:
                pass
        else:
            break
    cap.release()
    #cv2.destroyAllWindows()

In [96]:
video = "../opencv-np-workshop/data/video/Findet Nemo - Trailer.mp4"
template = "../opencv-np-workshop/data/img/nemo.jpg"

In [97]:
#video_detector(template,video,match_features_ratio)

In [98]:
video_detector(template,video,match_features_homography)